In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
df = pd.read_csv("./new_movie_data/final_cleaned_2.csv")

In [4]:
df.head()

,id,adult,belongs_to_collection,genres,overview,popularity,production_companies,release_date,runtime,spoken_languages,title,vote_average,vote_count,keywords,cast,crew
0,862,0,10194.0,"[16, 35, 10751]","Led by Woody, Andy's toys live happily in his ...",21.946943,[3],1995,81.0,[1],Toy Story,7.7,5415.0,"[931, 4290, 5202, 6054, 9713, 9823, 165503, 17...","[31, 12898, 7167, 12899, 12900, 7907, 8873, 11...","[7879, 12891, 7, 12892, 12893, 12894, 12895, 1..."
1,8844,0,NaN,"[12, 14, 10751]",When siblings Judy and Peter discover an encha...,17.015539,"[559, 2550, 10201]",1995,104.0,"[1, 2]",Jumanji,6.9,2413.0,"[10090, 10941, 15101, 33467, 158086, 158091]","[2157, 8537, 205, 145151, 5149, 10739, 58563, ...","[511, 876, 1729, 4945, 4951, 4952, 8023, 9967,..."
2,15602,0,119050.0,"[10749, 35]",A family wedding reignites the ancient feud be...,11.712900,"[6194, 19464]",1995,101.0,[1],Grumpier Old Men,6.5,92.0,"[1495, 12392, 179431, 208510]","[6837, 3151, 13567, 16757, 589, 16523, 7166]","[26502, 16837, 16837, 1551320]"
3,31357,0,NaN,"[35, 18, 10749]","Cheated on, mistreated and stepped on, the wom...",3.859495,[306],1995,127.0,[1],Waiting to Exhale,6.1,34.0,"[818, 10131, 14768, 15160, 33455]","[8851, 9780, 18284, 51359, 66804, 352, 87118, ...","[2178, 5144, 5144, 21968, 70592, 111118, 11111..."
4,11862,0,96871.0,[35],Just when George Banks has recovered from his ...,8.387519,"[5842, 9195]",1995,106.0,[1],Father of the Bride Part II,5.7,173.0,"[1009, 1599, 2246, 4995, 5600, 10707, 13149, 3...","[67773, 3092, 519, 70696, 59222, 18793, 14592,...","[37, 5506, 17698, 17698, 26160, 56106, 68755]"


In [5]:
df.iloc[0]

id                                                                     862
adult                                                                    0
belongs_to_collection                                              10194.0
genres                                                     [16, 35, 10751]
overview                 Led by Woody, Andy's toys live happily in his ...
popularity                                                       21.946943
production_companies                                                   [3]
release_date                                                          1995
runtime                                                               81.0
spoken_languages                                                       [1]
title                                                            Toy Story
vote_average                                                           7.7
vote_count                                                          5415.0
keywords                 

In [7]:
df.columns

Index(['id', 'adult', 'belongs_to_collection', 'genres', 'overview',
       'popularity', 'production_companies', 'release_date', 'runtime',
       'spoken_languages', 'title', 'vote_average', 'vote_count', 'keywords',
       'cast', 'crew'],
      dtype='object')

In [8]:
def score_similarity(a_,b_,attr):
    if attr == 'adult':
        return int(a_==b_) 
    elif attr == 'belongs_to_collection':
        if a_ == b_:
            return 1
        else:
            return 0
    elif attr in ['genres','production_companies','spoken_languages','keywords','crew','cast']:
        a_ = set(eval(a_))
        b_ = set(eval(b_))
        if len(a_|b_) == 0:
            return 0
        score = len(a_&b_)/len(a_|b_)
        return score
    elif attr in ['popularity','runtime','release_date','vote_average']:
        score = 1/abs(a_-b_)
        return score
    else:
        return 0

In [9]:
def get_similarity(a,b):
    a_ = df.iloc[df.index[df.id == a][0]]
    b_ = df.iloc[df.index[df.id == b][0]]

    similarity = []
    
    for i in list(a_.index)[1:]:
        similarity.append(score_similarity(a_[i],b_[i],i))

    return similarity


In [10]:
# don't consider vote count, title and overview
get_similarity(67758,8844)

[1,
 0,
 0.0,
 0,
 0.062063491945275884,
 0.0,
 0.125,
 0.07142857142857142,
 0.5,
 0,
 0.32258064516129026,
 0,
 0.0,
 0.0,
 0.0]